Mske sure to check if pytorch is available first

In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
import os
import numpy as np
import xml.etree.ElementTree as ET
import torchvision
from PIL import Image
from torchvision.models.detection import MaskRCNN
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor

In [3]:
torch.cuda.get_arch_list()

[]

In [2]:
print(torch.__version__)

2.2.2


In [3]:
print(torchvision.__version__)

0.17.2


In [4]:
def check_gpu_status():
    if torch.cuda.is_available():
        print("CUDA (GPU support) is available in PyTorch!")
        print(f"Number of GPU(s) available: {torch.cuda.device_count()}")
        for i in range(torch.cuda.device_count()):
            print(f"GPU {i}: {torch.cuda.get_device_name(i)}")
    else:
        print("CUDA (GPU support) is not available in PyTorch!")

check_gpu_status()


CUDA (GPU support) is not available in PyTorch!


In [6]:
print(torch.cuda.is_available())

False


In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
import os
import numpy as np
import xml.etree.ElementTree as ET
import torchvision
from PIL import Image
from torchvision.models.detection import MaskRCNN
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor

class KomiDataset(Dataset):
    def __init__(self, images_dir, annot_dir, transforms=None):

    def __getitem__(self, idx):
        img_name = self.filenames[idx]
        img_path = os.path.join(self.images_dir, img_name + '.jpg')
        annot_path = os.path.join(self.annot_dir, img_name + '.xml')
        
        img = Image.open(img_path).convert("RGB")
        
        boxes = []
        labels = []
        tree = ET.parse(annot_path)
        root = tree.getroot()
        
        for member in root.findall('object'):
            labels.append(1)  # Assuming 'face' is the only category
            
            bndbox = member.find('bndbox')
            xmin = int(bndbox.find('xmin').text)
            ymin = int(bndbox.find('ymin').text)
            xmax = int(bndbox.find('xmax').text)
            ymax = int(bndbox.find('ymax').text)
            boxes.append([xmin, ymin, xmax, ymax])
        
        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        labels = torch.as_tensor(labels, dtype=torch.int64)
        image_id = torch.tensor([idx])
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
        iscrowd = torch.zeros((len(boxes),), dtype=torch.int64)
        
        target = {}
        target["boxes"] = boxes
        target["labels"] = labels
        target["image_id"] = image_id
        target["area"] = area
        target["iscrowd"] = iscrowd
        
        if self.transforms is not None:
            img = self.transforms(img)
        
        return img, target


def get_model(num_classes):
    model = torchvision.models.detection.maskrcnn_resnet50_fpn(pretrained=True)
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
    
    in_features_mask = model.roi_heads.mask_predictor.conv5_mask.in_channels
    hidden_layer = 256
    model.roi_heads.mask_predictor = MaskRCNNPredictor(in_features_mask, hidden_layer, num_classes)
    
    return model

from torchvision.transforms import functional as F

def train_one_epoch(model, optimizer, data_loader, device, epoch):
    model.train()
    for images, targets in data_loader:
        images = list(F.to_tensor(image).to(device) for image in images)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
        
        loss_dict = model(images, targets)
        losses = sum(loss for loss in loss_dict.values())
        
        optimizer.zero_grad()
        losses.backward()
        optimizer.step()

def main():
    # Initialize the model, optimizer, and data loaders...
    model = get_model(num_classes=2)  # 1 class + background
    optimizer = torch.optim.SGD(model.parameters(), lr=0.005, momentum=0.9, weight_decay=0.0005)
    data_loader = DataLoader(KomiDataset(images_dir=r'C:\Users\brunolopez\Downloads\komi_analysis_pascal\JPEGImages', annot_dir=r'C:\Users\brunolopez\Downloads\komi_analysis_pascal\Annotations'), batch_size=2, shuffle=True)
    
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
    model.to(device)
    
    num_epochs = 10
    for epoch in range(num_epochs):
        train_one_epoch(model, optimizer, data_loader, device, epoch)
        # save your model periodically and evaluate if necessary

if __name__ == "__main__":
    main()
